In [1]:
!pip install nemo-toolkit[asr]

In [2]:
# ------------------------------
# 1. Funções auxiliares (ASR)
# ------------------------------
import librosa
import numpy as np
import time
import nemo.collections.asr as nemo_asr
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import torch

def carregar_audio(caminho, sr=16000):
    """Carrega arquivo de áudio e converte para 16 kHz mono."""
    audio, _ = librosa.load(caminho, sr=sr, mono=True)
    return np.array(audio, dtype=np.float32), sr

# ------------------------------
# Função para modelos NeMo
# ------------------------------
def avaliar_modelo_nemo(modelo_id, caminho_audio):
    """
    Executa inferência com modelos NeMo e retorna latência e transcrição.
    """
    print(f"🔽 Carregando modelo NeMo {modelo_id} ...")
    asr_model = nemo_asr.models.ASRModel.from_pretrained(modelo_id)

    inicio = time.time()
    transcricao_obj = asr_model.transcribe([caminho_audio])[0]
    fim = time.time()

    latencia_ms = (fim - inicio) * 1000
    transcricao_texto = getattr(transcricao_obj, "text", str(transcricao_obj))

    return latencia_ms, transcricao_texto

# ------------------------------
# Função para modelos Hugging Face
# ------------------------------
def avaliar_modelo_hf(modelo_id, caminho_audio):
    """
    Executa inferência com modelos Hugging Face (Wav2Vec2/Whisper) e retorna latência e transcrição.
    """
    print(f"🔽 Carregando modelo HF {modelo_id} ...")
    processor = Wav2Vec2Processor.from_pretrained(modelo_id)
    model = Wav2Vec2ForCTC.from_pretrained(modelo_id)
    model.eval()

    audio, sr = carregar_audio(caminho_audio, sr=16000)
    input_values = processor(audio, sampling_rate=sr, return_tensors="pt").input_values

    inicio = time.time()
    with torch.no_grad():
        logits = model(input_values).logits
    pred_ids = torch.argmax(logits, dim=-1)
    transcricao_texto = processor.batch_decode(pred_ids)[0]
    fim = time.time()

    latencia_ms = (fim - inicio) * 1000
    return latencia_ms, transcricao_texto

# ------------------------------
# Função para calcular métricas
# ------------------------------
def calcular_metricas(modelo, latencia, saida, referencia=None, rank=None):
    """
    Retorna dicionário com métricas: Rank, Modelo, Latência, Transcrição e WER aproximado.
    """
    # Garantir que a saída seja string
    if not isinstance(saida, str) and hasattr(saida, 'text'):
        saida = saida.text
    elif not isinstance(saida, str):
        saida = str(saida)

    resultado = {
        "Rank": rank if rank is not None else "-",
        "Modelo": modelo,
        "Latência (ms)": round(latencia, 2),
        "Transcrição": saida,
        "WER aproximado": None  # Sempre presente para evitar KeyError
    }

    if referencia:
        ref_tokens = referencia.lower().split()
        out_tokens = saida.lower().split()
        intersecao = len(set(ref_tokens) & set(out_tokens))
        wer_aprox = 1 - (intersecao / len(ref_tokens)) if ref_tokens else 1.0
        resultado["WER aproximado"] = round(wer_aprox, 3)

    return resultado



[NeMo W 2025-10-07 12:26:03 nemo_logging:405] /usr/local/lib/python3.12/dist-packages/pydub/utils.py:300: SyntaxWarning: invalid escape sequence '\('
      m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)
    
[NeMo W 2025-10-07 12:26:03 nemo_logging:405] /usr/local/lib/python3.12/dist-packages/pydub/utils.py:301: SyntaxWarning: invalid escape sequence '\('
      m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)
    
[NeMo W 2025-10-07 12:26:03 nemo_logging:405] /usr/local/lib/python3.12/dist-packages/pydub/utils.py:310: SyntaxWarning: invalid escape sequence '\('
      elif re.match('(flt)p?( \(default\))?$', token):
    
[NeMo W 2025-10-07 12:26:03 nemo_logging:405] /usr/local/lib/python3.12/dist-packages/pydub/utils.py:314: SyntaxWarning: invalid escape sequence '\('
      elif re.match('(dbl)p?( \(default\))?$', token):
    


In [ ]:

# ------------------------------
# Avaliação de modelos ASR específicos (PT-BR)
# ------------------------------

import os
import time
import pandas as pd
import librosa
import nemo.collections.asr as nemo_asr
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import torch
import numpy as np

# ------------------------------
# Funções auxiliares
# ------------------------------

def carregar_audio(caminho, sr=16000):
    """Carrega arquivo de áudio e converte para 16 kHz mono."""
    audio, _ = librosa.load(caminho, sr=sr, mono=True)
    return np.array(audio, dtype=np.float32), sr

def avaliar_modelo_nemo(modelo_id, caminho_audio):
    """Executa inferência com modelos NeMo (QuartzNet, Citrinet, FastConformer, Parakeet)."""
    print(f"\n🔹 Avaliando modelo NeMo: {modelo_id}")
    asr_model = nemo_asr.models.ASRModel.from_pretrained(modelo_id)

    inicio = time.time()
    transcricao_obj = asr_model.transcribe([caminho_audio])[0]
    fim = time.time()

    # Extrair texto se for objeto Hypothesis
    transcricao_texto = getattr(transcricao_obj, "text", str(transcricao_obj))
    latencia_ms = (fim - inicio) * 1000
    return latencia_ms, transcricao_texto

def avaliar_modelo_hf(modelo_id, caminho_audio):
    """Executa inferência com modelos Hugging Face (ex: wav2vec2)."""
    print(f"\n🔹 Avaliando modelo HF: {modelo_id}")
    processor = Wav2Vec2Processor.from_pretrained(modelo_id)
    model = Wav2Vec2ForCTC.from_pretrained(modelo_id)

    audio, sr = carregar_audio(caminho_audio)
    if sr != 16000:
        raise ValueError("O modelo HF requer áudio em 16 kHz.")

    inputs = processor(audio, sampling_rate=16000, return_tensors="pt", padding=True)
    inicio = time.time()
    with torch.no_grad():
        logits = model(inputs.input_values).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    fim = time.time()

    transcricao_texto = processor.batch_decode(predicted_ids)[0]
    latencia_ms = (fim - inicio) * 1000
    return latencia_ms, transcricao_texto

def calcular_metricas(modelo, latencia, saida, referencia=None):
    """Retorna métricas básicas para comparação de transcrição."""
    resultado = {
        "Modelo": modelo,
        "Latência (ms)": round(latencia, 2),
        "Transcrição": saida
    }

    if referencia:
        if not isinstance(saida, str):
            saida = str(saida)
        ref_tokens = referencia.lower().split()
        out_tokens = saida.lower().split()
        intersecao = len(set(ref_tokens) & set(out_tokens))
        wer_aprox = 1 - (intersecao / len(ref_tokens)) if ref_tokens else 1.0
        resultado["WER aproximado"] = round(wer_aprox, 3)

    return resultado

def limpar_transcricao(x):
    """Extrai texto se for objeto Hypothesis do NeMo."""
    try:
        if isinstance(x, (list, tuple)) and len(x) > 0:
            return getattr(x[0], "text", str(x[0]))
        return getattr(x, "text", str(x))
    except Exception:
        return str(x) if x is not None else ""

# ------------------------------
# Configurações
# ------------------------------

audio_teste = "/content/audio3.wav"          #/Caminho para o áudio
referencia_texto = "este é um exemplo de fala"

if not os.path.isfile(audio_teste):
    raise FileNotFoundError(f"Arquivo de áudio não encontrado: {audio_teste}")

# ------------------------------
# Lista de modelos para testar
# ------------------------------
modelos = [
     {"nome": "FastConformer-Hybrid", "id": "nvidia/stt_pt_fastconformer_hybrid_large_pc", "tipo": "nemo"},
     {"nome": "Citrinet-PT-Gamma-0.25", "id": "neongeckocom/stt_pt_citrinet_512_gamma_0_25", "tipo": "nemo"},
     {"nome": "QuartzNet-PT-Ottema", "id": "ottema/stt_pt_quartznet15x5_ctc_small", "tipo": "nemo"},
     {"nome": "QuartzNet-PT", "id": "dominguesm/stt_pt_quartznet15x5_ctc_small", "tipo": "nemo"},
     {"nome": "Parakeet-TDT-0.6b-v3 (multilingual)", "id": "nvidia/parakeet-tdt-0.6b-v3", "tipo": "nemo"},
     {"nome": "wav2vec2-PT-BR-Light", "id": "danielpedrozo/wav2vec2-portuguese-wpp-checkpoint-480", "tipo": "hf"},
]

# ------------------------------
# Execução e coleta de resultados
# ------------------------------
resultados = []

for modelo in modelos:
    try:
        if modelo["tipo"] == "nemo":
            latencia, transcricao = avaliar_modelo_nemo(modelo["id"], audio_teste)
        elif modelo["tipo"] == "hf":
            latencia, transcricao = avaliar_modelo_hf(modelo["id"], audio_teste)
        else:
            print(f"⚠️ Tipo de modelo desconhecido: {modelo['nome']}")
            continue
        metricas = calcular_metricas(modelo["nome"], latencia, transcricao, referencia_texto)
        resultados.append(metricas)
    except Exception as e:
        print(f"❌ Erro ao avaliar modelo {modelo['nome']}: {e}")

# ------------------------------
# Criação do DataFrame
# ------------------------------
df_resultados = pd.DataFrame(resultados)

if "Transcrição" in df_resultados.columns:
    df_resultados["Transcrição"] = df_resultados["Transcrição"].apply(limpar_transcricao)

# -------- ORDENAÇÃO --------
colunas_ordem = [c for c in ["WER aproximado", "Latência (ms)"] if c in df_resultados.columns]
if colunas_ordem:
    df_resultados = df_resultados.sort_values(
        by=colunas_ordem,
        ascending=[True] * len(colunas_ordem),
        na_position="last",
        ignore_index=True
    )

# Adicionar coluna de ranking
df_resultados.insert(0, "Rank", range(1, len(df_resultados) + 1))

# -------- VISUALIZAÇÃO NO CONSOLE --------
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_rows", None)

print("\n🔹 Resultados ordenados (melhor WER e menor latência):")

# Seleciona apenas colunas que realmente existem
colunas_display = [c for c in ["Rank", "Modelo", "WER aproximado", "Latência (ms)", "Transcrição"] if c in df_resultados.columns]
print(df_resultados[colunas_display])

# -------- EXPORTAÇÃO --------
saida_csv = "resultados_asr.csv"
saida_txt = "resultados_asr.txt"

df_resultados.to_csv(saida_csv, index=False, encoding="utf-8")

with open(saida_txt, "w", encoding="utf-8") as f:
    for _, row in df_resultados.iterrows():
        f.write(f"Modelo: {row.get('Modelo', 'N/A')}\n")
        f.write(f"WER aproximado: {row.get('WER aproximado', 'N/A')}\n")
        f.write(f"Latência (ms): {row.get('Latência (ms)', 'N/A')}\n")
        f.write(f"Transcrição: {row.get('Transcrição', '')}\n")
        f.write("-" * 60 + "\n")

print(f"\n✅ Resultados salvos em: {saida_csv} e {saida_txt}")


🔹 Avaliando modelo NeMo: nvidia/stt_pt_fastconformer_hybrid_large_pc


stt_pt_fastconformer_hybrid_large_pc.nem(…):   0%|          | 0.00/453M [00:00<?, ?B/s]

[NeMo W 2025-10-09 10:14:33 nemo_logging:405] Skipped conversion for config/subconfig:
    {'manifest_filepath': '???', 'sample_rate': 16000, 'batch_size': 32, 'shuffle': True, 'num_workers': 8, 'pin_memory': True, 'max_duration': 20, 'min_duration': 0.1, 'is_tarred': False, 'tarred_audio_filepaths': None, 'shuffle_n': 2048, 'bucketing_strategy': 'synced_randomized', 'bucketing_batch_size': None}
     Reason: Missing mandatory value: train_ds.manifest_filepath
        full_key: train_ds.manifest_filepath
        object_type=dict.
[NeMo W 2025-10-09 10:14:33 nemo_logging:405] Skipped conversion for config/subconfig:
    {'manifest_filepath': '???', 'sample_rate': 16000, 'batch_size': 32, 'shuffle': False, 'use_start_end_token': False, 'num_workers': 8, 'pin_memory': True}
     Reason: Missing mandatory value: validation_ds.manifest_filepath
        full_key: validation_ds.manifest_filepath
        object_type=dict.
[NeMo W 2025-10-09 10:14:33 nemo_logging:405] Skipped conversion for con

[NeMo I 2025-10-09 10:14:33 nemo_logging:393] Tokenizer SentencePieceTokenizer initialized with 128 tokens


[NeMo W 2025-10-09 10:14:33 nemo_logging:405] Skipped conversion for config/subconfig:
    {'manifest_filepath': '???', 'sample_rate': 16000, 'batch_size': 32, 'shuffle': True, 'num_workers': 8, 'pin_memory': True, 'max_duration': 20, 'min_duration': 0.1, 'is_tarred': False, 'tarred_audio_filepaths': None, 'shuffle_n': 2048, 'bucketing_strategy': 'synced_randomized', 'bucketing_batch_size': None}
     Reason: Missing mandatory value: train_ds.manifest_filepath
        full_key: train_ds.manifest_filepath
        object_type=dict.
[NeMo W 2025-10-09 10:14:33 nemo_logging:405] Skipped conversion for config/subconfig:
    {'manifest_filepath': '???', 'sample_rate': 16000, 'batch_size': 32, 'shuffle': False, 'use_start_end_token': False, 'num_workers': 8, 'pin_memory': True}
     Reason: Missing mandatory value: validation_ds.manifest_filepath
        full_key: validation_ds.manifest_filepath
        object_type=dict.
[NeMo W 2025-10-09 10:14:33 nemo_logging:405] Skipped conversion for con

...

vocab.json:   0%|          | 0.00/525 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/544 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.26G [00:00<?, ?B/s]


🔹 Resultados ordenados (melhor WER e menor latência):
   Rank                               Modelo  WER aproximado  Latência (ms)  \
0     1               Citrinet-PT-Gamma-0.25           0.333         592.23   
1     2                  QuartzNet-PT-Ottema           0.333         855.79   
2     3  Parakeet-TDT-0.6b-v3 (multilingual)           0.333        3514.44   
3     4                 FastConformer-Hybrid           0.333       16902.87   
4     5                 wav2vec2-PT-BR-Light           0.500        9936.41   
5     6                         QuartzNet-PT           0.667        1406.17   

                                                                         Transcrição  
0     um dois três testando isto é um exemplo para a transcrição de áudio para texto  
1     um dois três testando isto é um exemplo para a transcrição de audio para texto  
2  Um dois, três testando. Isto é um exemplo para a transcrição de áudio para texto.  
3   Um dos, três testando, isto é um exempl